## '''

Lionel Nurweze, start: 20240925, last change: 2024-10-28
 
lionel.nurweze -at- gmail -dot- com


Repository: https://github.com/nurweze/reportCreator


----------------------------------------------------------
References: 
Refactored using chatGPT OpenAI. (2023). 
ChatGPT (GPT4o - Sep 2024 version) [Large language model].
https://chat.openai.com/chat.

'''

In [ ]:
#!pip list
#''' Might need to install packages below '''
#!pip install pyreadr openpyxl pandas SQLAlchemy mysql-connector-python pymysql

In [ ]:
# Creating a necessary directory

#from tkinter import *
#from tkinter.ttk import Notebook
import os
import sys


'''
The program uses a directory ReportTemplates where standard reports are saved. 
We will be saving a working file "serialization_log.txt" here too. 
This cell makes sure that directory exists and if it doesn't creates it.
'''

# Find out current working directory 
print(f"Your working directory is for now {os.getcwd()}")

# Define the directory path
log_directory = 'ReportTemplates'

# Create the directory if it doesn't exist
os.makedirs(log_directory, exist_ok=True)

# Confirm that the directory ReportTemplates exists
print(f" '{log_directory}' directory is ready for use.")



**0. Repository and requirements** 

- Jupyterlab: Requires python(tkinter) or ipython kernels.

- Other required packages and their versions are specified in requirements.txt in the repository
  as well as imported in each cell where they're used. 


**1. Problem statement:** 

In professional life, on a regular basis, you might need to create the same (or a similar) report using different data sources. The usual issue is that the different data sources have been created at different time periods by different persons/teams for different purposes and saved in different formats with different variable names. This is the usual data inconsinstency problem which demands a lot of work into reading in files, changing variable names to make them match, etc. One other major problem is also to be able to read in large datafiles with unknown values.

If you are often producing the same report but from different data sources, it should be easy and pleasurable to produce the same report by making small adjustments. It should also be simple to create ad-hoc reports as well as manipulating data to produce the desired dataframe.


**2. Idea:** 

Adress the above problem by easing the report creation with automated serialization routines, and do that with help of a GUI. The end-goal is that the GUI could be bundled in a stand-alone program (executable), for those who don't want to delve in the coding parts. It is also a measure of security if one does not wish to temper with already functioning code.  

We use serialization to handle the different datatypes and sources. Serialization methods should solve the problem of handling large datasets.


**3. Scope:** 
 
The project started as part of a course DD1334 - Database Technology, at KTH, Stockholm. One of the course goals is to be able to use serialization techniques which is implemented here. Another goal is the use of query languages to model and structure data according to given constraints. Within the scope of the course the aim was just to produce a standard report with example data.

The author will continue to develop the program long-term and make it more stable and put it on a Github repository so that colleagues and others can reach it.
The future goals are stated below in Continued developments. 


 **4. Code architecture** 

The code exists in two formats that are both available on the Github repository: A Jupyter Notebook, and independent files. The latter is no longer being updated but the Jupyter Notebook follows the same structure, so that the new update code can be copied in the stand-alone files. 

The files functions are commented but the structure is basically the following: guimaker (.py) builds the GUI where the user sets the path to the datasource, the desired workspace and appropriate serialization method. importable (.py) identify the user inputs, identifies the file type, sets corresponding serialization method and is called by processes (.py) which contains the appropriate file-handling methods. serializable(.py) serializes or deserializes the data. exportable (.py) takes care of reading in the data and create the necessary report.  


**5. Implementation**

The program is able to handle input of .csv, .xlx/xls, JSON and SQL files and serialize them into .pickle or .json format and deserialize into a dataframe. 

Partly implemented are the handling of .xml, .r/.rdata and url:s. In order to handle URL files, for security reasons, you need to append your trusted source to a whitelist of sites.

**6. Usage**

In the GUI, the user adds a path to the datafile, confirms it, adds path to the workspace, confirms it, choose whether the file is going to be serialized with .pkl (Pickle) or with .json (JSON), and when that's done, the user presses the "save data to workspace" button. The data is then serialized in the right format (.pkl and/or .json) and with a filename that reminds of the original name as well as of the time of saving (while also making a serialization_log.txt). When it's for example files .csv or .xls files, the logic is simple, you read-in the file with the right method and serialize it (using both processes.py and importable.py). 

SQL methods are intricate so the decision was made to write a class of its own SQLHandler.py that handles those files. For now we're using a SQLite to handle MySQL/PostgreSQL databases syntaxes. Two alternatives were checked, the first one using SQLAlchemy to translate MySQL/PostgreSQL syntax to SQLite-readable logic but we've settled to using SQLAlchemy to point to built-in SQLite connections instead, which is a limiting factor in terms of which SQL syntax can be handled. Moreover, the logic for handling SQL files serialization is also different: We've decided to read-in the file as-is since there are already good packages (SQLite, SQLAlchemy) to handle those, running the SQL-program and serializing the results. 

There is a deserialization button, created in order to have an overview of the file since Pickle files are non-human readable. The user press the button and localize the .pkl file to deserialize.

Since the whole project revolves around the ease of creating reports, a "create report" button has been added, which the user presses to access a directory with standards reports to run, in form of Python files. The user can either use those reports or just call the report0.py to make his own file and tailor it according to their needs. 


**7. Continued developments** 

- Improve overall functionality, mostly when it comes to the report-creation logic.
- Implement SQLAlchemy which doesn't use SQLite engine to improve handling of more verbose MySQL/PostgreSQL files.  
- Improve JSON serialization that's still lacking.
- Have used pickle and json for serialization but should add a scrollbar to use other methods, python specific (parquet and feather seem promising) and others (probably  other javascript/json variants)
- Include API handling too, e.g. World Bank's and Eurostat's APIs.
- If code is regularly fetched from a website, implement data-scraping so that the report is generated automatically.  



In [ ]:
from IPython.display import HTML
video_id = 'bx4t0ItftDI?si=WUDGuUONevz7on7V'  # Extracted from your YouTube link
video_file_path = f'https://www.youtube.com/embed/{video_id}'
HTML(f"""
<iframe width="950" height="800" src="{video_file_path}" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" allowfullscreen></iframe>
""")

In [ ]:
# processes.py

import os
import logging

#from importable import FileReader, Serialization, Deserialization  # Adjust imports as necessary
#from sql_handler import SQLHandler  # Ensure to import your SQLHandler

# Configure logging for console
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

class FileProcessor:

    
    def __init__(self, confirmed_paths: dict, selected_format: str = "Pickle"):
        
        self.datasources = confirmed_paths.get("datasources", [])
        self.workspace = confirmed_paths.get("workspace")
        self.selected_format = selected_format
        self.file_reader = FileReader()
        self.serializer = Serialization()
        self.log_file_path = os.path.join("ReportTemplates", "serialization_log.txt")
        self.deserialization = Deserialization()

    
    def process_files(self) -> str:
        """
        Process each data source and serialize the content.
        """
        if not self.datasources or not self.workspace:
            logging.error("No paths were provided for data sources or workspace.")
            return "No paths were provided for data sources or workspace."
    
        result_messages = []
    
        for datasource in self.datasources:
            logging.info(f"Checking if data source exists: {datasource}")
    
            if not os.path.isfile(datasource):
                error_message = f"Data source does not exist: {datasource}"
                logging.error(error_message)
                result_messages.append(error_message)
                continue
    
            # Log the type of file being processed
            file_type = os.path.splitext(datasource)[1]
            logging.info(f"Processing file: {datasource} of type: {file_type}")
    
            try:
                logging.info(f"Reading data from: {datasource}")
    
                if datasource.endswith('.sql'):
                    logging.info(f"Detected SQL file: {datasource}")
                    sql_result_messages = self.process_SQL(datasource)
                    result_messages.extend(sql_result_messages)
                else:  # For non-SQL files, read and serialize 
                    data = self.file_reader.read_file(datasource)
                    if data is not None:
                        logging.info(f"Data read successfully from: {datasource}")
                        saved_file_path = self.serialize_data(data, datasource)
                        if saved_file_path:
                            result_messages.append(f"Serialized: {saved_file_path}")
                    else:
                        error_message = f"Failed to read data from: {datasource}"
                        logging.error(error_message)
                        result_messages.append(error_message)
    
            except Exception as e:
                error_message = f"An error occurred while processing {datasource}: {e}"
                logging.error(error_message)
                result_messages.append(error_message)
    
        return "\n".join(result_messages)

       

    def process_SQL(self, datasource: str) -> list:
        """
        Process and serialize the content of a SQL file.
        """
        result_messages = []
        logging.info(f"Processing SQL file: {datasource}")
    
        # Using SQLAlchemy's SQLite connection string format
        self.connection_string = f"sqlite:///{os.path.join(self.workspace, 'temp_database.db')}"
        logging.info(f"Connecting to database with connection string: {self.connection_string}")
        
        sql_handler = SQLHandler(workspace=self.workspace, serializer=self.serializer)
    
        try:
            # Read SQL commands from the file
            sql_commands = self.read_sql_file(datasource)
            if not sql_commands:
                error_message = f"No valid SQL commands found in: {datasource}"
                logging.error(error_message)
                return [error_message]
    
            for command in sql_commands:
                command = command.strip()  # Clean up whitespace
                if command:  # Skip empty commands
                    logging.info(f"Executing command: {command}")
                    try:
                        self.sql_handler.execute_sql(command)  # Assuming this method executes the SQL command
                    except Exception as e:
                        error_message = f"Error executing command '{command}': {e}"
                        logging.error(error_message)
                        result_messages.append(error_message)
    
            # Serialize results after execution
            saved_file_path = sql_handler.serialize_SQL(datasource, format=self.selected_format)
            if saved_file_path:
                result_messages.append(f"Serialized SQL Result: {saved_file_path}")
            else:
                error_message = f"No results were serialized for: {datasource}"
                logging.warning(error_message)
                result_messages.append(error_message)
    
        except Exception as e:
            error_message = f"Error processing SQL file {datasource}: {e}"
            logging.error(error_message)
            result_messages.append(error_message)
    
        return result_messages


    def read_sql_file(self, datasource: str) -> list:
        """Read and return the SQL commands from a file."""
        try:
            with open(datasource, 'r') as file:
                sql_commands = file.read().strip().split(';')  # Split by ';' to separate commands
            return [cmd for cmd in sql_commands if cmd]  # Filter out empty commands
        except Exception as e:
            logging.error(f"Failed to read SQL file {datasource}: {e}")
            return []

    
    def serialize_data(self, data, datasource: str) -> str:
        """Serialize the data and log the saved file path."""
        try:
            saved_file_path = self.serializer.serialize(
                data=data,
                workspace=self.workspace,
                format=self.selected_format,
                original_filename=datasource
            )
            with open(self.log_file_path, 'a') as log_file:
                log_file.write(f"Serialized: {saved_file_path}\n")
            logging.info(f"Serialized: {saved_file_path}")
            return saved_file_path
        except Exception as e:
            error_message = f"Error serializing data from {datasource}: {e}"
            logging.error(error_message)
            return ""


**IMPORTANT NOTE: On serialization of JSON and SQL files, and the file size problem.** 

1. For JSON files, there's usually no need to serialize them as byte/pickle-files to deserialize them afterwards, but that's also doable and could be practical if the user just needs to check the first lines of the file without opening the whole file. The user can just deserialize using the deserialize-button and the first lines are shown in console 2. When deserialized it's deserialized to JSON format again.

2. With regards to SQL files, as stated above, the author recommends to serialize to pickle and read the files as-is, since there already exist packages that handle SQL querying (SQLite -used here- and SQLAlchemy), run the query using those packages and reseliaze the results if needed maybe to JSON or again to pickle. 

    For smaller sized datasets it could be practical to directly parse them as JSON, but it's not recommended for very large datasets (or if there are many files) since DBMS are considerably faster. DBMS store the relationsships (keys) between the different datasets so it might be futile to lose that information just for the sake of a JSON serialization. The user can try both methods to see what works better.


3. Moreover, when it comes to SQL files, we're for now using the SQLite built-in package. The plan is that the code should be able to handle files with MySQL or PostgreSQL specific syntax and do that with the use of SQLAlchemy without the need to right own command. A temporarily method before full implementation is to use code translation, mapping MySQL/PostgreSQL specific syntaxes to code that functions with SQLite.   


In [ ]:
#importable.py


'''
Contains the classes FileReader, Serialization and Deserialization. 

FileReader reads in the file depending on its original datastructure so that information about the datastructure
is not lost under serialization.

Serialization class serializes the files according to the serialization of choice, as a byte file 
with Python's pickle or as a readable JSON format. Next serialization modules to implement will be Feather and Parquet. 

Implemented?|  Filetype      |  Original datastructure |               Deserialized data structure 
            |                |                         |   Pickle (Always python byte)      |  JSON
---------------------------------------------------------------------------------------------------------------
    YES     |     CSV        | Tables (Rows/Column)    | List of Dictionaries or pandas DF  | Str
----------------------------------------------------------------------------------------------------------------
    YES     |    Excel       | Tables (Rows/Column)    | List of Dictionaries or pandas DF  | Dictionary
----------------------------------------------------------------------------------------------------------------
    YES     |    JSON        | Dict. or list of Dict.  |               -----                | Kept as original
----------------------------------------------------------------------------------------------------------------
  NOT YET   |    XML         | Tables (Rows/Column)    | List of Dictionaries or pandas DF  | 
----------------------------------------------------------------------------------------------------------------
  NOT YET   |   R/RData      | Various (Vectors, Lists,| List of Dictionaries or pandas DF  |
            |                     DataFrames)          |                                    | 
----------------------------------------------------------------------------------------------------------------
   PARTLY   |    SQL (MySQL) |   MySQL code            | Keep as SQL code, use SQLite package |  .json (not fully implemented)     
                                                        (recommended for very large files)    | 
----------------------------------------------------------------------------------------------------------------
temporaly disabled |URL: http/https | see the different formats above         
----------------------------------------------------------------------------------------------------------------
 
'''

import os
import pandas as pd
import pickle
import xml.etree.ElementTree as ET
import pyreadr
import time
import json
import sqlite3
import re
import tkinter as tk
from tkinter import filedialog
from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker


class FileReader:


    def read_file(self, file_path: str):
        """
        Reads a file based on its extension and returns the data in an appropriate format.
        """
        data = None  # Initialization 
    
        # Getting the file extension
        _, file_extension = os.path.splitext(file_path)
        file_extension = file_extension.lower()
    
        # Check the file extension and use the appropriate reading method
        try:
            if file_extension == '.csv':
                with open(file_path, 'r') as file:
                    data = file.read()
                logging.info("Original file is a .CSV File.")
                
            elif file_extension == '.json':
                with open(file_path, 'r') as file:
                    data = json.load(file) 
                logging.info("Original file is a .JSON File.")
                         
            elif file_extension in ['.xls', '.xlsx']:
                data = pd.read_excel(file_path)  
                logging.info("Original file is an Excel File (.xls/.xlsx).")
                
            elif file_extension == '.xml':
                tree = ET.parse(file_path)
                root = tree.getroot()
                data = [{child.tag: child.text for child in elem} for elem in root]
                logging.info("XML File Content as List of Dictionaries.")                
            
            elif file_extension in ['.r', '.rdata']:
                result = pyreadr.read_r(file_path)
                data = result  # result is a dictionary-like object with dataframes
                logging.info("R File Content in Dictionary Format.")
        
            elif file_extension == '.sql':
                # For SQL files, read the content as a string (can be processed later)
                with open(file_path, 'r') as file:
                    data = file.read()
                logging.info("Original file is a .SQL File.")
    
            else:
                logging.error(f"Error: Unsupported file type: {file_extension}.")
                return None  # Indicate that the file type is unsupported
                
        except Exception as e:
            logging.error(f"Error reading file {file_path}: {e}")
            return None  # Indicate that an error occurred
    
        return data


class Serialization:

    
    def serialize(self, data, workspace, format, filename=None, original_filename=None):
        """
        Serialize data to the specified format and save it to the workspace.
        """
        # Validate the format before proceeding
        format = format.lower()
        valid_formats = {"pkl": "pickle", "pickle": "pickle", "json": "json"}
        if format not in valid_formats:
            raise ValueError(f"Unsupported serialization format '{format}'. Please choose 'Pickle' or 'JSON'.")

        # Standardize format to correct case for file naming
        format = valid_formats[format]

        # Check if the workspace path exists
        if not os.path.exists(workspace):
            raise FileNotFoundError(f"Workspace path '{workspace}' does not exist.")
    
        # Generate filename if not provided
        if not filename:
            if original_filename:
                base_name = os.path.splitext(os.path.basename(original_filename))[0]
                extension = "pkl" if format == "pickle" else "json"
                filename = f"{base_name}.{extension}"
            else:
                raise ValueError("Filename must be provided if original_filename is not given.")
    
        file_path = os.path.join(workspace, filename)
    
        try:
            # Serialize based on the selected format
            if format == "pickle":
                with open(file_path, 'wb') as pickled_file:
                    pickle.dump(data, pickled_file)
            elif format == "json":
                with open(file_path, 'w') as json_file:
                    json.dump(data, json_file, indent=4)
            logging.info(f"Data serialized successfully to {file_path}")
            return file_path  # Return only the path of the serialized file
    
        except Exception as e:
            logging.error(f"Error serializing data to {file_path}: {e}")
            raise RuntimeError(f"Error serializing data: {e}")


class Deserialization:
    
    '''
    Just focuses on the deserialization according to right datastructure. 
    The processing of logging and checking files is done by processes.py
    '''
    
    def __init__(self, os_module=os):
        self.os = os_module


    def deserialize_data(self, file_path):
        """Deserialize data from a given file based on its extension."""
        if not self.os.path.isfile(file_path):
            return f"Error: The file does not exist: {file_path}"
    
        _, file_extension = self.os.path.splitext(file_path)
        file_extension = file_extension.lower()
    
        try:
            if file_extension == '.pkl':
                with open(file_path, 'rb') as file:
                    data = pickle.load(file)
                    logging.info(f"Deserialized data from {file_path}: {data}")
                    return data
    
            elif file_extension == '.json':
                with open(file_path, 'r') as file:
                    data = json.load(file)
                    logging.info(f"Deserialized JSON data from {file_path}: {data}")
                    return data
    
            else:
                return f"Error: Unsupported file type: {file_extension}. Please provide a .pkl or .json file."
        except Exception as e:
            return f"Error during deserialization: {e}"



In [ ]:
# SQLHandler.py

import os
import pandas as pd
import logging
from sqlalchemy import create_engine, text
import sqlite3

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


class SQLHandler:

    
    def __init__(self, workspace, serializer):
        self.workspace = workspace
        self.serializer = serializer
        self.db_file_path = self._get_database_path()

        self.delete_temp_database()
        logging.info(f"Setting database file path to: {self.db_file_path}")
        self.ensure_database_exists()
        self.connection = sqlite3.connect(self.db_file_path)  # Direct SQLite connection
        self.engine = create_engine(f'sqlite:///{self.db_file_path}')  # SQLAlchemy engine for SQLite
        logging.info(f"Connected to the database at {self.db_file_path}")

    
    def _get_database_path(self):
        return os.path.join(self.workspace, "temp_database.db")

    
    def delete_temp_database(self):
        """Delete the temporary database file if it exists."""
        if os.path.isfile(self.db_file_path):
            os.remove(self.db_file_path)
            logging.info(f"Deleted existing database file: {self.db_file_path}")

            
    def ensure_database_exists(self):
        if not os.path.isfile(self.db_file_path):
            try:
                with sqlite3.connect(self.db_file_path) as conn:
                    logging.info(f"Database file created at {self.db_file_path}")
            except Exception as e:
                logging.error(f"Failed to create database file: {e}")
                raise RuntimeError(f"Failed to create database file: {e}")
        else:
            logging.info(f"Database file already exists at {self.db_file_path}")

    
    def read_sql(self, file_path):
        try:
            with open(file_path, 'r') as file:
                sql_commands = file.read()
                logging.info(f"Read SQL commands: {sql_commands}")  # Log the read commands
                return sql_commands
        except Exception as e:
            logging.error(f"Error reading SQL file: {e}")
            raise RuntimeError(f"Error reading SQL file: {e}")


    def identify_dialect(self, sql_commands):
        if "AUTO_INCREMENT" in sql_commands or "CREATE TABLE" in sql_commands:
            return 'mysql'
        elif "SERIAL" in sql_commands:
            return 'postgresql'
        return 'sqlite'  # Default to SQLite

    
    def _execute_with_sqlite(self, sql_commands):
        try:
            commands = sql_commands.strip().split(';')
            for command in commands:
                command = command.strip()
                if command:
                    logging.info(f"Executing SQL command: {command}")
                    result = self.connection.execute(command)
                    logging.info(f"Executed SQL command: {command}. Affected rows: {result.rowcount}")  
            self.connection.commit()  
            logging.info("All changes committed to the database.")  
        except Exception as e:
            logging.error(f"SQLite execution failed: {e}")
            raise

    
    def execute_sql(self, sql_commands):
        dialect = self.identify_dialect(sql_commands)
        logging.info(f"Identified SQL dialect: {dialect}")
        
        try:
            if dialect == 'sqlite':
                logging.info("Executing SQL commands in SQLite dialect.")
                self._execute_with_sqlite(sql_commands)
            else:
                logging.warning(f"Dialect {dialect} not supported; executing with SQLite as a fallback.")
                self._execute_with_sqlite(sql_commands)  # Fallback to SQLite
    
            # Return a success message if execution reaches here
            return "SQL commands executed successfully."
        
        except Exception as e:
            # Log the error and return an error message
            logging.error(f"SQL execution failed: {e}")
            return f"Error executing SQL commands: {e}"

    
    def get_all_data(self):
        try:
            results = {}
            table_names = self._get_all_table_names()
            if not table_names:
                logging.warning("No tables found in the database to fetch data from.")
                return results
            
            logging.info(f"Fetching data from tables: {table_names}")
            for (table_name,) in table_names:
                try:
                    results[table_name] = pd.read_sql_table(table_name, con=self.engine)
                    logging.info(f"Fetched data from '{table_name}': {results[table_name]}")
                except Exception as e:
                    logging.error(f"Error reading data from table '{table_name}': {e}") 
            return results
        except Exception as e:
            logging.error(f"Error fetching data: {e}")
            raise


    def _get_all_table_names(self):
        return self.connection.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()

    
    def serialize_SQL(self, datasource: str, format: str) -> str:
        """Execute SQL commands from a file and serialize the results."""
        result_messages = []
        
        try:
            with open(datasource, 'r') as sql_file:
                sql_commands = sql_file.read()
            
            self.execute_sql(sql_commands)  # Execute SQL commands
            
            # Fetch all data after execution
            results = self.get_all_data()  # Ensure this returns a valid DataFrame
            logging.info(f"Results after executing SQL: {results}")  # Log fetched results
            
            if results is not None and not results.empty:
                saved_file_path = self.serializer.serialize(
                    data=results,
                    workspace=self.workspace,
                    format=format,
                    original_filename=datasource
                )
                result_messages.append(f"Serialized SQL Result: {saved_file_path}")
            else:
                result_messages.append("No results to serialize from executed SQL.")
        
        except Exception as e:
            logging.error(f"Error in serialize_SQL: {e}")
            result_messages.append(f"Error processing SQL file {datasource}: {e}")
    
        return result_messages



In [ ]:
#GUIMaker.py

import tkinter as tk
from tkinter import filedialog, messagebox
import os
import datetime

#import importable

#from SQLHandler import SQLHandler

class GUIMaker:

    
    def __init__(self, master):
        self.master = master
        self.master.title("Data Processor")
        self.data_path = ""
        self.workspace_path = ""
        self.serialization_format = tk.StringVar(value="Pickle")  # Default to Pickle format
        self.sql_handler = SQLHandler()
        self.file_reader = importable.FileReader()  # Instance of FileReader
        self.serializer = importable.Serialization()  # Instance of Serialization

        # Setup GUI layout
        self.setup_gui()

    
    def setup_gui(self):
        # Path Entry for Data Source
        tk.Label(self.master, text="Data Source Path:").grid(row=0, column=0, sticky="e")
        self.data_entry = tk.Entry(self.master, width=50)
        self.data_entry.grid(row=0, column=1)
        tk.Button(self.master, text="Browse", command=self.browse_data).grid(row=0, column=2)

        # Path Entry for Workspace
        tk.Label(self.master, text="Workspace Path:").grid(row=1, column=0, sticky="e")
        self.workspace_entry = tk.Entry(self.master, width=50)
        self.workspace_entry.grid(row=1, column=1)
        tk.Button(self.master, text="Browse", command=self.browse_workspace).grid(row=1, column=2)

        # Serialization Format Option
        tk.Label(self.master, text="Select Serialization Format:").grid(row=2, column=0, sticky="e")
        format_options = ["Pickle", "JSON"]
        self.format_menu = tk.OptionMenu(self.master, self.serialization_format, *format_options)
        self.format_menu.grid(row=2, column=1, sticky="w")

        # Save Data Button
        tk.Button(self.master, text="Save Data", command=self.save_data_to_workspace).grid(row=3, column=1, pady=10)

    
    def browse_data(self):
        file_path = filedialog.askopenfilename(title="Select Data File")
        if file_path:
            self.data_entry.delete(0, tk.END)
            self.data_entry.insert(0, file_path)
            self.data_path = file_path

    
    def browse_workspace(self):
        directory = filedialog.askdirectory(title="Select Workspace Directory")
        if directory:
            self.workspace_entry.delete(0, tk.END)
            self.workspace_entry.insert(0, directory)
            self.workspace_path = directory

    
    def save_data_to_workspace(self):
        # Retrieve paths and format selection
        self.data_path = self.data_entry.get()
        self.workspace_path = self.workspace_entry.get()
        format_choice = self.serialization_format.get().lower()
    
        # Input validation
        if not self.data_path or not self.workspace_path:
            messagebox.showerror("Error", "Please specify both data source and workspace paths.")
            return
    
        # Determine original filename and log filename for confirmation
        original_filename = os.path.basename(self.data_path)
        log_filename = f"{os.path.splitext(original_filename)[0]}_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}"
    
        # Process the file based on type
        _, file_extension = os.path.splitext(self.data_path)
        file_extension = file_extension.lower()
    
        try:
            if file_extension == ".sql":
                result_messages = []
                if "mysql" in self.data_path.lower():
                    result_messages = self.sql_handler.process_sqlalchemy(self.data_path)
                else:
                    result_messages = self.sql_handler.process_sqlite(self.data_path)
    
                # Debugging log
                logging.info(f"SQL processing results: {result_messages}")
    
                # Check for success message in results
                if not result_messages or any("Error" in msg for msg in result_messages):
                    messagebox.showerror("Error", "Failed to process SQL file. " + "\n".join(result_messages))
                    return
                
                # Serialize SQL result
                serialized_file_path = self.serializer.serialize(result_messages, self.workspace_path, format_choice,
                                                                 filename=f"{log_filename}.{format_choice}",
                                                                 original_filename=original_filename)
    
            else:
                # Process non-SQL files
                data = self.file_reader.read_file(self.data_path)
                serialized_file_path = self.serializer.serialize(data, self.workspace_path, format_choice,
                                                                 filename=f"{log_filename}.{format_choice}",
                                                                 original_filename=original_filename)
    
            messagebox.showinfo("Success", f"Data saved to {serialized_file_path}")
    
        except Exception as e:
            messagebox.showerror("Error", f"Failed to save data: {e}")


In [ ]:
#exportable.py

import os
import logging
import io
import re
import sys
import tkinter as tk
import pandas as pd


from tkinter import filedialog
'''
import Serialization, Deserialization
import FileProcessor
'''

''' 
Important note: that the ReportTemplates directory is hardcoded as 
the default map for all reports. 
'''

class Exportable: 


    def __init__(self, gui = None):
        self.gui = gui  # Reference to the GUIMaker instance
        self.log_file_path = os.path.join("ReportTemplates", "serialization_log.txt")
        

    def create_report(self):
        """Open a Python file for editing and running."""
        file_path = self.get_python_file_path()  # Open file dialog to get the path of the report
        if not file_path:
            self.gui.log_to_console("Error: No file path provided for the report.")
            return
    
        with open(file_path, 'r') as file:
            report_code = file.read()
    
        # Open a new window for editing the Python script
        editor_window = tk.Toplevel(self.gui.window)
        editor_window.title(f"Edit Report - {os.path.basename(file_path)}")  # Display the file name in the window title
    
        # Create a text editor widget to display the content of the Python report
        editor_text = tk.Text(editor_window, height=20, width=80)
        editor_text.insert(tk.END, report_code)  # Insert the contents of the report
        editor_text.pack()
    
        # Define a button to run the report, passing the file path to run_report
        run_button = tk.Button(
            editor_window,
            text="Run Report",
            command=lambda: self.run_report(editor_text.get("1.0", tk.END), report_file_path=file_path)
        )
        run_button.pack()


    def get_python_file_path(self):
        """Open a file dialog to select a Python file."""
        file_path = filedialog.askopenfilename(
            title="Select a Python file",
            filetypes=[("Python files", "*.py"), ("All files", "*.*")]
        )
        return file_path


    def run_report(self, report_code, report_file_path=None):
        """Execute the report code with the proper file path and log messages to console 1 and output to console 2."""
        output_buffer = io.StringIO()
        error_buffer = io.StringIO()
        sys.stdout = output_buffer  # Redirect stdout to capture print statements
        sys.stderr = error_buffer   # Redirect stderr to capture errors
    
        # Ensure ReportTemplates is added to sys.path only once
        report_dir = os.path.join(os.path.dirname(__file__), "ReportTemplates")
        if report_dir not in sys.path:
            sys.path.append(report_dir)
    
        # Change directory to ReportTemplates
        os.chdir(report_dir)
    
        # Create a custom globals dictionary with __file__ set to the provided report file path
        exec_globals = {"__file__": report_file_path} if report_file_path else {}
    
        try:
            # Run the provided report code with the modified globals context
            exec(report_code, exec_globals)
    
            # Capture the standard output and error output
            output = output_buffer.getvalue()
            error_output = error_buffer.getvalue()
    
            # Log outputs accordingly
            if output:
                self.gui.log_to_console_2(output)  # Log stdout to console 2
            if error_output:
                self.gui.log_to_console(f"Errors detected:\n{error_output}")  # Log stderr to console 1
    
            # Handle cases where no output is produced
            if not output and not error_output:
                self.gui.log_to_console_2("No output produced.")
    
        except Exception as e:
            # Capture unexpected exceptions in a more readable format
            self.gui.log_to_console(f"Error executing report: {e}")
        finally:
            # Reset stdout and stderr back to their original streams
            sys.stdout = sys.__stdout__
            sys.stderr = sys.__stderr__


    def read_serialization_log(self):
        """Read the serialization log and return the list of serialized or deserialized file paths."""
        log_file_path = os.path.join(os.path.dirname(__file__), "ReportTemplates", "serialization_log.txt")

        if not os.path.isfile(log_file_path):
            logging.error("Serialization log does not exist.")
            return []

        # Debugging: Print the log file path
        print(f"Reading log file at: {log_file_path}")

        with open(log_file_path, 'r') as log_file:
            lines = log_file.readlines()

        # Print log file contents for debugging
        print("Log file contents:")
        for line in lines:
            print(line.strip())

        # Extract file paths from log
        file_paths = []
        for line in lines:
            line = line.strip()
            if "Serialized:" in line or "Deserialized:" in line:
                parts = line.split(":")
                if len(parts) > 1:
                    file_path = parts[1].strip()
                    file_paths.append(file_path)
                    print(f"Extracted file path: '{file_path}'")

        if not file_paths:
            logging.error("No file paths found in the serialization log.")
        
        return file_paths


    def log_deserialization(self, file_path):
        """Log the deserialized file path to the serialization log."""
        with open(self.log_file_path, 'a') as log_file:
            log_file.write(f"Deserialized: {file_path}\n")


class ReportGenerator:
    
    '''
    Contains methods that collect the datapaths of processed files. 
    Is called by report0.py to produce standard report. 
    '''

    def __init__(self):
        self.logger = logging.getLogger()
        self.deserializer = Deserialization()  # Initialize once
        self.current_dir = os.path.dirname(os.path.abspath(__file__))

    
    def generate_report(self):
        self.logger.info("Starting report generation...")
        self.logger.info(f"Current directory: {self.current_dir}")
    
        # Initialize Exportable to read the serialization log
        exportable_instance = Exportable()
        log_file_path = os.path.join(self.current_dir, "serialization_log.txt")
    
        try:
            # Read the serialization log to get file paths
            file_paths = exportable_instance.read_serialization_log()
    
            self.logger.info(f"File paths from log: {file_paths}")
    
            if not file_paths:
                self.logger.error("No file paths found in the serialization log.")
                return
    
            # Process each file path to deserialize and inspect contents
            for file_path in file_paths:
                data = self.deserializer.deserialize_data(file_path)  # Use the correct method
                
                if isinstance(data, str) and data.startswith("Error"):
                    self.logger.error(data)  # Log errors from deserialization
                else:
                    # Log or print summary information about the deserialized data
                    if isinstance(data, pd.DataFrame):
                        self.logger.info(f"Successfully deserialized DataFrame from {file_path}.\n: Shape:{data.shape}. Preview: {data.head()}")
                    elif isinstance(data, dict):
                        self.logger.info(f"Successfully deserialized JSON from {file_path}.:\n Keys:{list(data.keys())}")
                    else:
                        self.logger.info(f"Successfully deserialized data from {file_path}:\n{data}")  # Adjust this line if data needs special formatting
    
            self.logger.info("Report generation completed.")
            return True  # Indicate success
    
        except Exception as e:
            self.logger.error(f"An error occurred in generate_report: {e}")
            return False  # Indicate failure


In [ ]:
##############################################################
## Testfile for SQL methods  without needing to run the GUI ##
##############################################################

'''
Keep this test-file here to test for SQL files. Could also be deleted. 
For other methods, use the GUI, it's simpler. 
'''
from datetime import datetime
import logging

# Initialize serializer, SQLHandler, and Deserialization with workspace path
serializer = Serialization()
deserializer = Deserialization()  # Create an instance of Deserialization

## Use for example: workspace = '/home/.../Desktop'
workspace = os.getcwd() # Just for this case, we set up the current directory

sql_handler = SQLHandler(workspace, serializer)

# Step 3: Read SQL Commands using the instance of SQLHandler

### Use for example: file_path = '/home/.../Datasets/scriptDB2.sql'
testdata_directory = os.path.join(os.getcwd(), 'Testdata')
file_path = os.path.join(testdata_directory, 'scriptDB2.sql')
sql_commands = sql_handler.read_sql(file_path)  # Call read_sql on the instance

# Step 4: Execute SQL Commands
sql_handler.execute_sql(sql_commands)

# Step 5: Fetch and Print Data
results = sql_handler.get_all_data()
print(results)

# Step 6: Serialize Data (Optional)
# Define the filename for the serialized data
serialized_file_name = f"{file_path.split('/')[-1].replace('.sql', '')}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.pkl"

# Serialize the results
serializer.serialize(results, workspace, 'pkl', filename=serialized_file_name, original_filename=file_path)
print(f"Data serialized to {serialized_file_name}")

# Deserialization Test
# Attempt to deserialize the serialized file
deserialized_data = deserializer.deserialize_data(f"{workspace}/{serialized_file_name}")
print("Deserialized Data:", deserialized_data)


In [ ]:
# main.py
import logging
from tkinter import messagebox
from GUImaker import GUIMaker  # Ensure this line is uncommented

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


def main():
    """Main function to run the GUI and process files."""
    # Initialize and run the GUI
    try:
        app = GUIMaker()  # Make sure GUIMaker is correctly implemented to accept necessary parameters
        app.start()
    except Exception as e:
        logging.error(f"An error occurred while starting the application: {e}")
        messagebox.showerror("Error", f"An error occurred: {e}")
        
if __name__ == "__main__":
    main()
